<a href="https://colab.research.google.com/github/Deelacole2/Final_Project/blob/feature%2FDatabase/Data_load_to_RDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [55.5 kB]
Hit:13 http://ppa.launchpad.net/cran

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-05-08 01:16:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.48MB/s    in 0.2s    

2021-05-08 01:16:30 (5.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://yellowteambucket.s3.us-east-2.amazonaws.com/hosp_addr.csv"
spark.sparkContext.addFile(url)
hosp_addr_df = spark.read.csv(SparkFiles.get("hosp_addr.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
hosp_addr_df.show()

+---+--------------------+---------------+-----+-----+-----------+-----------+-------------------+
|fid|             address|           city|state|  zip|     county|   latitude|          longitude|
+---+--------------------+---------------+-----+-----+-----------+-----------+-------------------+
|  2|      150 VIA MERIDA|WESTLAKE VILAGE|   CA|91362|    VENTURA|34.15493887|       -118.8157364|
|  3|4060 WHITTIER BOU...|    LOS ANGELES|   CA|90023|LOS ANGELES| 34.0236473|       -118.1841648|
|  4|6245 DE LONGPRE A...|      HOLLYWOOD|   CA|90028|LOS ANGELES|34.09639136|-118.32523490000001|
|  5|14148 FRANCISQUIT...|   BALDWIN PARK|   CA|91706|LOS ANGELES|34.06303893|       -117.9674378|
|  6|3700 EAST SOUTH S...|       LAKEWOOD|   CA|90712|LOS ANGELES|33.85970662|        -118.148403|
|  7|323 SOUTH HELIOTR...|       MONROVIA|   CA|91016|LOS ANGELES|34.14875866|-117.99266770000001|
|  8|900 SOUTH ATLANTI...|  MONTEREY PARK|   CA|91754|LOS ANGELES|34.04946606|       -118.1382622|
| 10| 730 

In [ ]:
url="https://yellowteambucket.s3.us-east-2.amazonaws.com/hosp_closures.csv"
spark.sparkContext.addFile(url)
hosp_closures_df = spark.read.csv(SparkFiles.get("hosp_closures.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
hosp_closures_df.show()

+--------------------+--------------------+-----------+-----+-----+-------+----------------+--------------+-------------+------------+
|            hospital|             address|       city|state|  zip|   cbsa|medicare_payment|number_of_beds|closure_month|closure_year|
+--------------------+--------------------+-----------+-----+-----+-------+----------------+--------------+-------------+------------+
|Jellico Medical C...|   188 HOSPITAL LANE|    JELLICO|   TN|37762|  Metro|             PPS|            54|     November|        2020|
|Perry Community H...|2718 Squirrel Hol...|     Linden|   TN|37096|Neither|             PPS|            25|     November|        2020|
|Acoma-Canonocito-...|   80B Veterans Blvd|      Acoma|   NM|87034|  Micro|             IHS|             6|     November|        2020|
|Northridge Medica...|70 Medical Center Dr|   Commerce|   GA|30529|  Micro|             PPS|            27|      October|        2020|
|Southwest Georgia...|     361 Randolph St|   Cuthbert|

In [ ]:
url="https://yellowteambucket.s3.us-east-2.amazonaws.com/hosp_metric.csv"
spark.sparkContext.addFile(url)
hosp_metric_df = spark.read.csv(SparkFiles.get("hosp_metric.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
hosp_metric_df.show()

+--------------------+---------+-------+---------------+-----------------+------------------+------------------+
|               state|stateabbr| deaths|population_2019|deaths_per_100000|death_to_pop_ratio|avg_cost_of_living|
+--------------------+---------+-------+---------------+-----------------+------------------+------------------+
|        Alabama (01)|       AL| 54,108|      4,903,185|         1,103.50|             1.10%|             89.33|
|         Alaska (02)|       AK|  4,613|        731,545|            630.6|             0.63%|              99.4|
|        Arizona (04)|       AZ| 60,236|      7,278,717|            827.6|             0.83%|             99.69|
|       Arkansas (05)|       AR| 32,888|      3,017,804|         1,089.80|             1.09%|              88.3|
|     California (06)|       CA|269,831|     39,512,223|            682.9|             0.68%|            126.27|
|       Colorado (08)|       CO| 39,390|      5,758,736|              684|             0.68%|   

In [ ]:
import pandas as pd

hosp_metric_df = hosp_metric_df.withColumn("population_2019", hosp_metric_df.population_2019.cast('int'))
hosp_metric_df = hosp_metric_df.withColumn("deaths_per_100000", hosp_metric_df.deaths_per_100000.cast('int'))
hosp_metric_df = hosp_metric_df.withColumn("deaths", hosp_metric_df.deaths.cast('int'))


In [ ]:
hosp_metric_df.dtypes

[('state', 'string'),
 ('stateabbr', 'string'),
 ('deaths', 'int'),
 ('population_2019', 'int'),
 ('deaths_per_100000', 'int'),
 ('death_to_pop_ratio', 'string'),
 ('avg_cost_of_living', 'double')]

In [ ]:
url="https://yellowteambucket.s3.us-east-2.amazonaws.com/hospital_popup.csv"
spark.sparkContext.addFile(url)
hospital_popup_df = spark.read.csv(SparkFiles.get("hospital_popup.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
hospital_popup_df.show()

+---+--------------------+---------------+-----+-----------+------+------------------+
|fid|                name|           city|state|     county|status|              type|
+---+--------------------+---------------+-----+-----------+------+------------------+
|  2|LOS ROBLES HOSPIT...|WESTLAKE VILAGE|   CA|    VENTURA|  OPEN|GENERAL ACUTE CARE|
|  3|EAST LOS ANGELES ...|    LOS ANGELES|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
|  4|SOUTHERN CALIFORN...|      HOLLYWOOD|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
|  5|KINDRED HOSPITAL ...|   BALDWIN PARK|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
|  6|LAKEWOOD REGIONAL...|       LAKEWOOD|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
|  7|MONROVIA MEMORIAL...|       MONROVIA|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
|  8|MONTEREY PARK HOS...|  MONTEREY PARK|   CA|LOS ANGELES|  OPEN|GENERAL ACUTE CARE|
| 10|CORONA REGIONAL M...|         CORONA|   CA|  RIVERSIDE|  OPEN|GENERAL ACUTE CARE|
| 11|MODOC MEDICAL CENTER|        ALTURAS| 

In [ ]:
# Loading the svi data

url="https://yellowteambucket.s3.us-east-2.amazonaws.com/svi_data.csv"
spark.sparkContext.addFile(url)
svi_data_df = spark.read.csv(SparkFiles.get("svi_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
svi_data_df.show()

+-------+-------+----------+----+--------------------+-----------------+------+---------+
|  state|st_abbr|    county|fips|            location|        area_sqmi|   svi|total_pop|
+-------+-------+----------+----+--------------------+-----------------+------+---------+
|ALABAMA|     AL|   Autauga|1001|Autauga County, A...|      594.4434589|0.4354|    55200|
|ALABAMA|     AL|    Blount|1009|Blount County, Al...|644.8304602000001|0.4242|    57645|
|ALABAMA|     AL|    Butler|1013|Butler County, Al...|      776.8382008|0.8653|    20025|
|ALABAMA|     AL|   Calhoun|1015|Calhoun County, A...|      605.8672514|0.8252|   115098|
|ALABAMA|     AL|  Chambers|1017|Chambers County, ...|596.5606429999999|0.7382|    33826|
|ALABAMA|     AL|    Coffee|1031|Coffee County, Al...|678.9858207999999|0.5408|    51288|
|ALABAMA|     AL|   Colbert|1033|Colbert County, A...|592.7446865000001|0.4274|    54495|
|ALABAMA|     AL| Covington|1039|Covington County,...|      1030.556053|0.7723|    37351|
|ALABAMA| 

In [ ]:
url="https://yellowteambucket.s3.us-east-2.amazonaws.com/us_hospital_locations.csv"
spark.sparkContext.addFile(url)
raw_hospital_data = spark.read.csv(SparkFiles.get("us_hospital_locations.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
raw_hospital_data.show()

+--------------+-----------+---+--------+--------------------+--------------------+---------------+-----+-----+-------------+--------------+------------------+------+----------+-----------+----------+-------+-----------+------------+----------+--------------------+--------------------+---------------+-------------+--------------+--------------------+-------------+--------------------+-------+--------------------+---------+----+-------------+-------+
|             X|          Y|FID|      ID|                NAME|             ADDRESS|           CITY|STATE|  ZIP|         ZIP4|     TELEPHONE|              TYPE|STATUS|POPULATION|     COUNTY|COUNTYFIPS|COUNTRY|   LATITUDE|   LONGITUDE|NAICS_CODE|          NAICS_DESC|              SOURCE|     SOURCEDATE|   VAL_METHOD|      VAL_DATE|             WEBSITE|     STATE_ID|            ALT_NAME|ST_FIPS|               OWNER|TTL_STAFF|BEDS|       TRAUMA|HELIPAD|
+--------------+-----------+---+--------+--------------------+--------------------+---------

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://capstone.cjj05msruqqh.us-east-2.rds.amazonaws.com:5432/capstone_db"
config = {"user":"postgres", 
          "password": "yellow_team", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to hosp_addr_df table in RDS

hosp_addr_df.write.jdbc(url=jdbc_url, table='hosp_addr', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS

hosp_closures_df.write.jdbc(url=jdbc_url, table='hosp_closures', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

hosp_metric_df.write.jdbc(url=jdbc_url, table='state_metrics', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

svi_data_df.write.jdbc(url=jdbc_url, table='svi_data', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS

hospital_popup_df.write.jdbc(url=jdbc_url, table='hosp_popup', mode=mode, properties=config)

In [ ]:
# Write dataframe to raw hospital data in RDS

raw_hospital_data.write.jdbc(url=jdbc_url, table='raw_hospital_data', mode=mode, properties=config)